In [1]:
#Загрузка необходимых библиотек
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
# Загрузка данных, удаление лишних данных по всем 3 выборкам
column=['0takt','1takt','2takt','3takt','4takt','5takt','6takt','7takt','8takt',
                          '9takt','10takt','11takt','12takt','13takt','14takt','15takt','16takt','17takt']
try:
    df1=pd.read_csv('SOP1.dat',delimiter=' ',encoding='windows-1250',header=None,error_bad_lines=False)

    df2=pd.read_csv('SOP2.dat',delimiter=' ',encoding='windows-1250',header=None,error_bad_lines=False)
    df3=pd.read_csv('SOP3.dat',delimiter=' ',encoding='windows-1250',header=None,error_bad_lines=False)
    print('Данные загруженны ')
except:
    print('Данные не загрузились ')

try:  
    df1=df1.drop([0,0]) # удаление технической строки
    df1=df1.drop(columns=[17,0]) # удаление лишних столбцов,   расстояние мм будет индекс
    df2=df2.drop([0,0]) # удаление технической строки
    df2=df2.drop(columns=[17,0])  # удаление лишних столбцов,   расстояние мм будет индекс 
    df3=df3.drop([0,0]) # удаление технической строки
    df3=df3.drop(columns=[17,0])# удаление лишних столбцов,   расстояние мм будет индекс
    df1=df1.astype("float32")# приводим данные к float
    df2=df2.astype("float32")# приводим данные к float
    df3=df3.astype("float32")# приводим данные к float
    print('Данные  обработаны ')
except:
    print('Данные не обработались ')



Данные загруженны 
Данные  обработаны 


C:\Users\tai-sku\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Приведение к одной координате

In [3]:
df11=df1.copy()

#Приведение к общей координате
for n in range(1,17):
    df11[n]=np.roll(df11[n],-50)
    
for p in [3,4,5,6,9,10]:
    df11[n]=np.roll(df11[n],95)
    
for n in range(1,17):
    df11[n]=np.roll(df11[n],-95)
    
#Создание датафреймов по стороне и по типу деффекта    
df11A=df11[[1,11,13,3,9,5]].copy()
df11B=df11[[2,12,14,4,10,6]].copy()
df11P=df11[[7,8,15,16]].copy()

In [ ]:
df11.plot.line(subplots=True)

In [ ]:
#Выбор переменных для определения неспллошности на стороне А,В и Р -  и создание датафрейма

df11A.plot.line(subplots=True)

In [ ]:

df11B.plot.line(subplots=True)

In [ ]:

df11P.plot.line(subplots=True)

# Добавим целевой вектор

In [8]:
#Загрузка файла с целевой переменной
targ=pd.read_table('target1.csv',sep=',')
targ
targ.fillna(0)

,Начало дефекта,Длина дефекта,Высота дефекта,Тип дефекта,Сторона
0,51,42.0,7,L,A
1,200,40.0,6,L,B
2,278,39.0,4,L,A
3,365,43.0,6,L,B
4,460,27.0,5,L,A
5,845,36.0,4,L,B
6,959,62.0,6,L,A
7,141,0.0,4,T,


In [ ]:
# создадим вектор целевой
df11A['targ']=0
df11B['targ']=0
df11P['targ']=0

In [ ]:
#Добавим значение целевого вектора в каждом датафрейме
for n in range(len(targ)):
    if targ['Сторона'][n]=='A':
        df11A.targ[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n]-1+int(targ['Длина дефекта'][n]))]=1
    if targ['Сторона'][n]=='B':
        df11B.targ[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n]-1+int(targ['Длина дефекта'][n]))]=1  
    if targ['Тип дефекта'][n]=='T':
        df11P.targ[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n])]=1

In [ ]:
df11B.plot.line(subplots=True)

In [ ]:
#Сделаем в датафрейме общем разбитие на классы в соответствии с имеющимся таргетом
df11['deffect']=0
for n in range(len(targ)):
    if targ['Сторона'][n]=='A':
        df11.deffect[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n]-1+int(targ['Длина дефекта'][n]))]=1
    if targ['Сторона'][n]=='B':
        df11.deffect[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n]-1+int(targ['Длина дефекта'][n]))]=2  
    if targ['Тип дефекта'][n]=='T':
        df11.deffect[targ['Начало дефекта'][n]-1:(targ['Начало дефекта'][n])]=3

In [ ]:
df11.plot.line(subplots=True)

# Подберем алгоритм кластеризации и сравним результаты кластеризации с таргетом

In [ ]:
df11.plot.line(subplots=True)

In [ ]:
from sklearn import cluster as cl
from sklearn.preprocessing import StandardScaler

In [ ]:
SC=StandardScaler()

In [ ]:
X=SC.fit_transform(df11)

In [ ]:


# создадим пустой список для записи показателя WCSS (нашей ошибки)
wcss = []

# воспользуемся  функцией range(), она возвращает последовательность чисел
# в данном случае числа от 1 до 10
for i in range(1, 20):
    
    # настроим параметры модели (подробнее об этом на сайте)
    kmeans = cl.KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 42)

    # обучим модель на наших данных с разным количеством кластеров
    kmeans.fit(df11)

    # для каждого кластера рассчитаем ошибку (атрибут inertia_) и поместим в список
    wcss.append(kmeans.inertia_)

# зададим размер график
plt.figure(figsize = (10,6))
# передадим функции plot() последовательность кластеров и ошибку WCSS
plt.plot(range(1, 20), wcss)
# и добавим подписи
plt.title('Выбор количества кластеров методом локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')

In [ ]:
#Для метода К-среднее оптимальное количество кластеров 6 или 8

In [ ]:
# Используем несколько алгоритмов кластеризации
KM=cl.KMeans(n_clusters=6,  init='k-means++', n_init=10, max_iter=300, 
          tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto')
DB=cl.DBSCAN(eps=50,  min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', 
       leaf_size=30, p=None, n_jobs=None)
AP=cl.AffinityPropagation( damping=0.5, max_iter=200, convergence_iter=15, copy=True,
                    preference=None, affinity='euclidean', verbose=False, random_state=None)

AC=cl.AgglomerativeClustering(n_clusters=6,  affinity='euclidean', memory=None, connectivity=None, 
                        compute_full_tree='auto', linkage='ward', distance_threshold=None, compute_distances=False)

OP=cl.OPTICS( min_samples=5, max_eps=50, metric='minkowski', p=2, metric_params=None, cluster_method='xi', 
       eps=None, xi=0.05, predecessor_correction=True, min_cluster_size=None, algorithm='auto', 
       leaf_size=30, memory=None, n_jobs=None)

df111=df11.copy()
df111['KmeansSC']=KM.fit_predict(X)
df111['DBSCANSC']=DB.fit_predict(X)
df111['APSC']=DB.fit_predict(X)
df111['ACSC']=AC.fit_predict(X)
df111['OPSC']=OP.fit_predict(X)

df111['Kmeans']=KM.fit_predict(df11)
df111['DBSCAN']=DB.fit_predict(df11)
df111['AP']=AP.fit_predict(df11)
df111['AC']=AC.fit_predict(df11)
df111['OP']=OP.fit_predict(df11)

In [ ]:
# посмотрим на целевую переменную с помощью точечной диаграммы

# зададим размер
plt.figure(figsize = (10,6))

# возьмем первый и второй столбцы с помощью функции iloc()
# параметр с задаст каждой точке определенный код (0, 1, 2), а
# camp определит цвет каждой из точек
plt.scatter(df111.loc[:,1], df111.loc[:,11], c = df111.targ, cmap = 'Paired')

In [ ]:
# матрица рассеяния и гистограммы
from pandas.plotting import scatter_matrix
scatter_matrix(df11, alpha=0.1);

In [ ]:
df111.plot.line(subplots=True)

# Для всех тактов общая класстеризация не очень интуитивна. Cделаем классификацию для каждой стороны и типа деффекта

In [7]:
df11=df1.copy()

#Приведение к общей координате
for n in range(1,17):
    df11[n]=np.roll(df11[n],-50)
    
for p in [3,4,5,6,9,10]:
    df11[n]=np.roll(df11[n],95)
    
for n in range(1,17):
    df11[n]=np.roll(df11[n],-95)
    
#Создание датафреймов по стороне и по типу деффекта    
df11A=df11[[1,11,13,3,9,5]].copy()
df11B=df11[[2,12,14,4,10,6]].copy()
df11P=df11[[7,8,15,16]].copy()

In [ ]:





# создадим пустой список для записи показателя WCSS (нашей ошибки)
wcss = []

# воспользуемся  функцией range(), она возвращает последовательность чисел
# в данном случае числа от 1 до 10
for i in range(1, 20):
    
    # настроим параметры модели (подробнее об этом на сайте)
    kmeans = cl.KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 42)

    # обучим модель на наших данных с разным количеством кластеров
    kmeans.fit(df11A)

    # для каждого кластера рассчитаем ошибку (атрибут inertia_) и поместим в список
    wcss.append(kmeans.inertia_)

# зададим размер график
plt.figure(figsize = (10,6))
# передадим функции plot() последовательность кластеров и ошибку WCSS
plt.plot(range(1, 20), wcss)
# и добавим подписи
plt.title('Выбор количества кластеров методом локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')

Количество кластеров 7 для К среднего

In [ ]:
X=SC.fit_transform(df11A)
X1=df11A
# Используем несколько алгоритмов кластеризации
KM=cl.KMeans(n_clusters=6,  init='k-means++', n_init=10, max_iter=300, 
          tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto')
DB=cl.DBSCAN(eps=50,  min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', 
       leaf_size=30, p=None, n_jobs=None)
AP=cl.AffinityPropagation( damping=0.5, max_iter=200, convergence_iter=15, copy=True,
                    preference=None, affinity='euclidean', verbose=False, random_state=None)

AC=cl.AgglomerativeClustering(n_clusters=6,  affinity='euclidean', memory=None, connectivity=None, 
                        compute_full_tree='auto', linkage='ward', distance_threshold=None, compute_distances=False)

OP=cl.OPTICS( min_samples=5, max_eps=50, metric='minkowski', p=2, metric_params=None, cluster_method='xi', 
       eps=None, xi=0.05, predecessor_correction=True, min_cluster_size=None, algorithm='auto', 
       leaf_size=30, memory=None, n_jobs=None)

df111A=df11A.copy()
df111A['KmeansSC']=KM.fit_predict(X)
#df111A['DBSCANSC']=DB.fit_predict(X)
#df111A['APSC']=DB.fit_predict(X)
df111A['ACSC']=AC.fit_predict(X)
df111A['OPSC']=OP.fit_predict(X)

df111A['Kmeans']=KM.fit_predict(X1)
df111A['DBSCAN']=DB.fit_predict(X1)
df111A['AP']=AP.fit_predict(X1)
df111A['AC']=AC.fit_predict(X1)
df111A['OP']=OP.fit_predict(X1)

df111A['targ']=df11A.targ
y=df11A.targ
#Добавим значение целевого вектора в каждом датафрейме

df111A.plot.line(subplots=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=50) 

X_train.shape, y_train.shape
X_test.shape, y_test.shape

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

#  Метрики качества модели
print('Основные метрики качества на test:')
print("Accuracy: %.2f" % metrics.accuracy_score(y_pred, y_test))
print("Precision: %.2f" % metrics.precision_score(y_pred, y_test, average='macro'))
print("Recall: %.2f" % metrics.recall_score(y_pred, y_test, average='macro'))
print("F1: %.2f" % metrics.f1_score(y_pred, y_test, average='macro'))
print()

print('Матрица ошибок:')
print(metrics.confusion_matrix(y_pred, y_test))